In [1]:
!nvcc --version

!wget https://antidote.cloud/f/ae5312aa983845c7abf1/?dl=1 -O llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
--2026-01-15 14:16:44--  https://antidote.cloud/f/ae5312aa983845c7abf1/?dl=1
Resolving antidote.cloud (antidote.cloud)... 193.30.122.219
Connecting to antidote.cloud (antidote.cloud)|193.30.122.219|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://antidote.cloud/seafhttp/files/f44a2c96-b68f-4538-ac71-21c3a33a971a/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl [following]
--2026-01-15 14:16:45--  https://antidote.cloud/seafhttp/files/f44a2c96-b68f-4538-ac71-21c3a33a971a/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl
Reusing existing connection to antidote.cloud:443.
HTTP request sent, awaiting response... 200 OK
Length: 52058581 (50M) [application/octet-stream]
Saving to: ‘llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl’

llama_c

In [2]:
#!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama_cpp_python
!CMAKE_ARGS="-DGGML_CUDA=on" pip install /content/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl
!pip install huggingface_hub hf_transfer
%env HF_HUB_ENABLE_HF_TRANSFER=1
!huggingface-cli download bartowski/Mistral-Nemo-Instruct-2407-GGUF Mistral-Nemo-Instruct-2407-Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False


Processing ./llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
env: HF_HUB_ENABLE_HF_TRANSFER=1
/usr/local/lib/python3.12/dist-packages/huggingface_hub/commands/download.py:141: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Mistral-Nemo-Instruct-2407-Q5_K_M.gguf: 100% 8.73G/8.73G [01:00<00:00, 144MB/s]
Download complete. Moving file to Mistral-Nemo-Instruct-2407-Q5_K_M.gguf
Mistral-Nemo-Instruct-2407-Q5_K_M.gguf


In [32]:
!huggingface-cli download bartowski/Mistral-Nemo-Instruct-2407-GGUF \
Mistral-Nemo-Instruct-2407-Q5_K_M.gguf \--local-dir .--local-dir-use-symlinks False

usage: huggingface-cli <command> [<args>]
huggingface-cli: error: unrecognized arguments: False


In [5]:
from llama_cpp import Llama
llm = Llama(
  model_path="./Mistral-Nemo-Instruct-2407-Q5_K_M.gguf",
  n_ctx=2048, # Context window (tokens)
  n_gpu_layers=35, # All layers on GPU
  n_threads=8, # CPU threads
  n_batch=512, # Batch size
  verbose=False,
  seed=1234 # Reproducibility!
)

llama_context: n_ctx_per_seq (2048) < n_ctx_train (1024000) -- the full capacity of the model will not be utilized


In [6]:
prompt = "[INST] What is the capital of Germany? [/INST]"
output = llm(prompt, max_tokens=50, temperature=0)
print(output["choices"][0]["text"])

The capital of Germany is Berlin. It has been the capital since the reunification of Germany in 1990, when the former West German capital, Bonn, and the former East German capital, East Berlin, were united. Berlin is known


In [7]:
from llama_cpp import LlamaGrammar
import json
schema = {
  "type": "object",
  "properties": {
    "sentiment": {
      "type": "string",
      "enum": ["positive", "negative", "neutral"]
    },
    "stars": {
      "type": "string",
      "enum": ["1", "2", "3", "4", "5"]
    }
  },
  "required": ["sentiment", "stars"]
}
grammar = LlamaGrammar.from_json_schema(json.dumps(schema))

In [8]:
review = "This product is absolutely terrible. Waste of money."
prompt = f"[INST] {review} [/INST]"
output = llm(prompt, max_tokens=50, temperature=0, grammar=grammar)
result = json.loads(output["choices"][0]["text"])
print(result) # {"sentiment": "negative", "stars": "1"}

{'sentiment': 'negative', 'stars': '1'}


In [11]:
import kagglehub
import pandas as pd
path = kagglehub.dataset_download(
"ashishkumarak/amazon-shopping-reviews-daily-updated"
)
df = pd.read_csv(path + "/amazon_reviews.csv")
results = []
for idx, row in df.head(50).iterrows():
  prompt = f"[INST] {row['content']} [/INST]"
  output = llm(prompt, max_tokens=50, temperature=0, grammar=grammar)
  result = json.loads(output["choices"][0]["text"])
  result["original_stars"] = row["score"]
  results.append(result)

results_df = pd.DataFrame(results)

Using Colab cache for faster access to the 'amazon-shopping-reviews-daily-updated' dataset.


In [12]:
schema = {
  "type": "object",
  "properties": {
    "sentiment": {
      "type": "string",
      "enum": ["positive", "negative", "neutral"]
    },
    "topic": {
      "type": "string",
      "enum": ["delivery", "price", "quality", "service", "other"]
    },
    "mentions_competitor": {
      "type": "string",
      "enum": ["yes", "no"]
    },
    "actionable": {
      "type": "string",
      "enum": ["yes", "no"]
    }
  },
  "required": ["sentiment", "topic", "mentions_competitor", "actionable"]
}

In [23]:
text = "This product is good."

In [20]:
text = "The delivery was incredibly fast and the product quality exceeded my expectations. Highly recommend!"

In [28]:
text = "This product broke after only a week of use. Absolutely terrible quality and a waste of money. I'm very disappointed."

In [14]:
prompt = f"[INST] {text} [/INST]"

In [16]:
prompt = f"""[INST] You are analyzing customer reviews for an
e-commerce platform. Classify the sentiment and star rating:
Review: {text} [/INST]"""

In [30]:
prompt = f"""[INST] Classify customer reviews:
Examples:
- "Love it! Perfect!"-> positive, 5 stars
- "Terrible. Broke immediately."-> negative, 1 star
- "Decent product for the price"-> neutral, 3 stars

Now classify: {text} [/INST]"""

In [15]:
for chunk in llm(prompt, max_tokens=100, temperature=0, grammar=grammar, stream=True):
  print(chunk["choices"][0]["text"], end="", flush=True)

{"sentiment": "positive", "stars": "5"}

In [17]:
for chunk in llm(prompt, max_tokens=100, temperature=0, grammar=grammar, stream=True):
  print(chunk["choices"][0]["text"], end="", flush=True)

{"sentiment": "positive", "stars": "4"}

In [31]:
for chunk in llm(prompt, max_tokens=100, temperature=0, grammar=grammar, stream=True):
  print(chunk["choices"][0]["text"], end="", flush=True)

{"sentiment": "negative", "stars": "1"}